In [2]:
import tensorflow as tf
#from PIL import Image
#import matplotlib.pylab as plt
import numpy as np

if int(tf.__version__[0]) < 2:
    tf.enable_eager_execution()

Download COCO dataset and distill visualwakewords for the interested classes

In [ ]:
! python3 download_and_convert_data \
    --dataset_name='visualwakewords' \
    --dataset_dir='visualwakewords_vehicle' \
    --small_object_area_threshold=0.05 \
    --download=False \
    --foreground_class_of_interest=['bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat'] \

In [3]:
dataset_dir = 'visualwakewords_person'
train_filenames = [dataset_dir + '/train.record-000{:02d}-of-00100'.format(i) for i in range(100)]
val_filenames  = [dataset_dir + '/val.record-0000{:01d}-of-00010'.format(i) for i in range(10)]

In [4]:
train_dataset = tf.data.TFRecordDataset(train_filenames)
val_dataset = tf.data.TFRecordDataset(val_filenames)

In [5]:
feature_description = {
  'image/height': tf.io.FixedLenFeature([], tf.int64),
  'image/width': tf.io.FixedLenFeature([], tf.int64),
  'image/filename': tf.io.FixedLenFeature([], tf.string),
  'image/source_id': tf.io.FixedLenFeature([], tf.string),
  'image/key/sha256': tf.io.FixedLenFeature([], tf.string),
  'image/encoded': tf.io.FixedLenFeature([], tf.string),
  'image/format': tf.io.FixedLenFeature([], tf.string),
  'image/class/label': tf.io.FixedLenFeature([], tf.int64),
  #'image/object/bbox/xmin': tf.io.FixedLenFeature([], tf.float32),
  #'image/object/bbox/xmax': tf.io.FixedLenFeature([], tf.float32),
  #'image/object/bbox/ymin': tf.io.FixedLenFeature([], tf.float32),
  #'image/object/bbox/ymax': tf.io.FixedLenFeature([], tf.float32),
  #'image/object/area': tf.io.FixedLenFeature([], tf.float32)
  }
def _parse_image_function(example_proto):
  # Parse the input tf.Example proto using the dictionary above.
  return tf.io.parse_single_example(example_proto, feature_description)

parsed_train_dataset = train_dataset.map(_parse_image_function)
parsed_val_dataset = val_dataset.map(_parse_image_function)

In [6]:
train_labels = []
for example in parsed_train_dataset:
    train_labels.append(int(example['image/class/label']))

Instructions for updating:
Colocations handled automatically by placer.


In [7]:
val_labels = []
for example in parsed_val_dataset:
    val_labels.append(int(example['image/class/label']))

In [8]:
print('Total number of sample for train: {} val: {}'.format(len(train_labels), len(val_labels)))
print('Total number of images with vehicle in train: {} val: {}'.format(sum(train_labels), sum(val_labels)))

Total number of sample for train: 82783 val: 40504
Total number of images with vehicle in train: 26476 val: 12674


In [17]:
import json 

with open('./visualwakewords_person/visualwakewords/annotations/instances_train2014.json') as f:
    coco_annotations_dict = json.load(f)


In [18]:
for i,elem in enumerate(coco_annotations_dict['images']):
    if elem['file_name'] == 'COCO_train2014_000000445327.jpg':
    #if i>10:
    #    break
        print(elem)
        id = elem['id']
        objects = []
        for ann in coco_annotations_dict['annotations']:
            if ann['image_id'] == id:
                objects.append(ann['category_id'])
        print(objects)
        categories = []
        for obj in objects:
            for categ in coco_annotations_dict['categories']:
                if categ['id'] == obj:
                    categories.append(categ['name'])
        print(categories)

{'license': 1, 'file_name': 'COCO_train2014_000000445327.jpg', 'coco_url': 'http://images.cocodataset.org/train2014/COCO_train2014_000000445327.jpg', 'height': 480, 'width': 640, 'date_captured': '2013-11-19 20:30:31', 'flickr_url': 'http://farm8.staticflickr.com/7210/6815871618_23be7190bf_z.jpg', 'id': 445327}
[8, 1, 1, 1, 52, 52, 52, 52, 52, 52, 52, 52, 52, 52, 52, 52, 1, 52, 52]
['truck', 'person', 'person', 'person', 'banana', 'banana', 'banana', 'banana', 'banana', 'banana', 'banana', 'banana', 'banana', 'banana', 'banana', 'banana', 'person', 'banana', 'banana']
